### **Integer Programming**

In [1]:
import os, sys, copy

import numpy as np
import pandas as pd

from tabulate import tabulate
from fractions import Fraction

from helper import visualize_tabulation, dual_simplex_to_all_slack_starting, all_slack_starting

In [20]:
# fistly, we have to define a function to add a new constraint to an existing tabulation
def add_constraint(tabulation, constraint, new_var, all_vars, basic_vars):

    tabulation = copy.deepcopy(tabulation)
    all_vars = copy.deepcopy(all_vars)
    basic_vars = copy.deepcopy(basic_vars)

    # adding a new row for the basic slack variable
    tabulation = np.concatenate((tabulation, constraint.reshape(1, -1)))

    # adding a new column
    column = np.zeros(tabulation.shape[0])
    column[-1] = 1
    column = np.array(list(map(Fraction, column)))
    temp = tabulation[:, range(tabulation.shape[1]-1)]
    temp = np.concatenate((temp, column.reshape(-1, 1)), axis=1)
    tabulation = np.concatenate((temp, tabulation[:, -1].reshape(-1, 1)), axis=1)

    # add the new variable
    all_vars.append(new_var)
    basic_vars.append(new_var)

    return tabulation, all_vars, basic_vars

### Define the Problem

In [3]:
# no. of decision variables (N)
NO_OF_DECISION_VARS = 3

# no. of conditions 
NO_OF_CONDITIONS = 2

# --------------------------------- objective function ---------------------------------
# specify the objective function in the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>]
OBJ_FUNC = [2, 20, -10]

# -------------------------------- type of optimization --------------------------------
OPT_TYPE = "MAX" # or "MIN" 

# ------------------------------------- conditions -------------------------------------
# specify each condition in a new row with the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>, 'operation', <solution>]
# the operation can be one of the '>=', '<=', or '='
conditions = [
    [   2,  20,   4, '<=',  15],
    [   6,  20,   4,  '=',  20],
]

# define M value
M = 1e6

In [4]:
# create the variables and generate the initial tabulation
OBJ_FUNC = np.array(OBJ_FUNC)
conditions = np.array(conditions)

# extract the operations
operations = list(conditions[:, -2])
no_of_less_than_or_equals    = operations.count('<=')
no_of_greater_than_or_equals = operations.count('>=')
no_of_equalities             = operations.count('=')

# find the size of a row
row_size = 1 + NO_OF_DECISION_VARS + no_of_less_than_or_equals + 2*no_of_greater_than_or_equals + no_of_equalities + 1

# find the column size
col_size = 2 + NO_OF_CONDITIONS # no. of basic vars = 2 (for objective function) + no. of conditions

# initialize the initial tabulation
initial_tabulation = np.zeros((col_size, row_size), dtype=object)

var_symbol_arr = []
basic_var_symbol_arr = []

OBJ_FUNC_VAR_SYMBOL = "P"
DECISION_VAR_SYMBOL = "X"
SLACK_VAR_SYMBOL    = "S"
SURPLUS_VAR_SYMBOL  = "S"
ARTIFICIAL_VAR_SYMBOL = "A"

var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

# create the objective function row
basic_var_symbol_arr.extend([OBJ_FUNC_VAR_SYMBOL] * 2)
initial_tabulation[0][0] = 1
initial_tabulation[0][1 : len(OBJ_FUNC)+1] = -OBJ_FUNC

for i in range(NO_OF_DECISION_VARS):
    var_symbol_arr.append(DECISION_VAR_SYMBOL + str(i+1))

# create all the other basic variable rows
index = 1 + NO_OF_DECISION_VARS
artificial_row_idxs = []
for i in range(NO_OF_CONDITIONS):

    initial_tabulation[i+2][1 : NO_OF_DECISION_VARS+1] = conditions[i][ :-2]
    initial_tabulation[i+2][-1] = conditions[i][-1]
    operation = operations[i]

    if operation == '<=':
        # introduce a slack variable (a basic variable)
        var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))
        initial_tabulation[i+2][index] = 1
        index += 1
        basic_var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))
    elif operation == ">=":
        # introduce a surplus  (a non-basic variable) and an artificial variable (a basic variable) 
        var_symbol_arr.append(SURPLUS_VAR_SYMBOL + str(i+1))
        initial_tabulation[i+2][index] = -1
        index += 1
        var_symbol_arr.append(ARTIFICIAL_VAR_SYMBOL + str(i+1))
        initial_tabulation[i+2][index] =  1
        initial_tabulation[1][index] = -1 if OPT_TYPE == 'MIN' else 1 # carefull!!!
        index += 1
        basic_var_symbol_arr.append(ARTIFICIAL_VAR_SYMBOL + str(i+1))
        artificial_row_idxs.append(i+2)
    elif operation == "=":
        # introduce an artificial variable (a basic variable)
        var_symbol_arr.append(ARTIFICIAL_VAR_SYMBOL + str(i+1))
        initial_tabulation[i+2][index] = 1
        initial_tabulation[1][index] = -1 if OPT_TYPE == 'MIN' else 1 # careful!!!
        index += 1
        basic_var_symbol_arr.append(ARTIFICIAL_VAR_SYMBOL + str(i+1))
        artificial_row_idxs.append(i+2)

print("variables:", var_symbol_arr)
print("basic variables:", basic_var_symbol_arr)

# convert all the elements into fractions
for i in range(initial_tabulation.shape[0]):
    initial_tabulation[i] = np.array(list(map(Fraction, initial_tabulation[i])))

visualize_tabulation(initial_tabulation, var_symbol_arr, basic_var_symbol_arr, big_M=True)

variables: ['P', 'X1', 'X2', 'X3', 'S1', 'A2']
basic variables: ['P', 'P', 'S1', 'A2']
┌────┬─────┬──────┬──────┬──────┬──────┬──────┬───────┐
│    │   P │   X1 │   X2 │   X3 │   S1 │  A2  │   sol │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ P  │   1 │   -2 │  -20 │   10 │    0 │  1M  │     0 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │    2 │   20 │    4 │    1 │  0   │    15 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ A2 │   0 │    6 │   20 │    4 │    0 │  1   │    20 │
└────┴─────┴──────┴──────┴──────┴──────┴──────┴───────┘


In [5]:
# make the artificial variable columns pivot vectors
for artificial_row_idx in artificial_row_idxs:
    if OPT_TYPE == "MIN": initial_tabulation[1] += initial_tabulation[artificial_row_idx]
    else:                 initial_tabulation[1] -= initial_tabulation[artificial_row_idx]

print("="*20 + " Initial Feasible Solution " + "="*20)
visualize_tabulation(initial_tabulation, var_symbol_arr, basic_var_symbol_arr, big_M=True)

==================== Initial Feasible Solution ====================
┌────┬─────┬───────┬─────────┬───────┬──────┬──────┬───────┐
│    │   P │  X1   │   X2    │  X3   │   S1 │   A2 │  sol  │
├────┼─────┼───────┼─────────┼───────┼──────┼──────┼───────┤
│ P  │   1 │ -2-6M │ -20-20M │ 10-4M │    0 │    0 │ -20M  │
├────┼─────┼───────┼─────────┼───────┼──────┼──────┼───────┤
│ S1 │   0 │   2   │   20    │   4   │    1 │    0 │  15   │
├────┼─────┼───────┼─────────┼───────┼──────┼──────┼───────┤
│ A2 │   0 │   6   │   20    │   4   │    0 │    1 │  20   │
└────┴─────┴───────┴─────────┴───────┴──────┴──────┴───────┘


In [6]:
iteration_no = 0
tabulation = copy.deepcopy(initial_tabulation)

for _ in range(10): # *****

    # ====================================== ENTERING VARIABLE ======================================
    obj_func_row = tabulation[0] + tabulation[1] * M

    if OPT_TYPE == "MAX":
        if not (obj_func_row[1:-1] < 0).any():
            # raise Exception(f"there are no negative values in the objective row...")
            print(f"***** there are no more negative values in the objective row; tabulation is optimal... *****\n")
            break
        pivot_col_value = np.min(obj_func_row[1:-1]) # highest negative value
    else: # OPT_TYPE == "MIN"
        if not (obj_func_row[1:-1] > 0).any():
            # raise Exception(f"there are no positive values in the objective row...")
            print(f"***** there are no more positive values in the objective row; tabulation is optimal... *****\n")
            break
        pivot_col_value = np.max(obj_func_row[1:-1]) # highest positive value

    iteration_no += 1
    print("="*20 + f" iteration no.: {iteration_no} " + "="*20)

    pivot_col_idx = list(obj_func_row[1:-1]).index(pivot_col_value) + 1
    # -----------------------------------------------------------------------------------------------

    entering_var = var_symbol_arr[pivot_col_idx]
    print(f"entering var: {entering_var}")
    pivot_col = tabulation[:, pivot_col_idx]
    print(f"pivot col: {pivot_col}")

    # find the ratio column
    solution  = tabulation[:, -1]
    print(f"solution : {solution}")
    ratio_col = np.float64(solution) / np.float64(pivot_col)
    print(f"ratio col: {ratio_col}")

    # ====================================== LEAVING VARIABLE =======================================
    # find the lowest positive value in the ratio column
    if not (ratio_col[2: ] > 0).any():
        raise Exception(f"there is no positive value in the ratio column; therefore, NO FEASIBLE SOLUTION EXISTS...")

    pivot_row_value = np.min(ratio_col[2: ][ratio_col[2: ] > 0])
    pivot_row_idx = list(ratio_col[2: ]).index(pivot_row_value) + 2
    # -----------------------------------------------------------------------------------------------

    leaving_var = basic_var_symbol_arr[pivot_row_idx]
    print(f"leaving var : {leaving_var}")
    pivot_row = tabulation[pivot_row_idx]
    print(f"pivot row: {pivot_row}")

    # replace the leaving variable with the entering variable
    basic_var_symbol_arr[pivot_row_idx] = entering_var
    print(f"new basic variables: {basic_var_symbol_arr}")

    # convert the pivot column to a pivot vector

    # verify that the pivot column is convertible to a pivot vector
    if pivot_row[pivot_col_idx] == 0:
        raise Exception(f"the pivot column (var '{entering_var}') at index {pivot_col_idx} is not convertible to a pivot vector...")

    # transform the pivot row
    pivot_row_ = pivot_row / pivot_row[pivot_col_idx]
    tabulation[pivot_row_idx] = pivot_row_

    # transform all the other rows
    for i in range(col_size):

        if i == pivot_row_idx:
            continue

        row = tabulation[i]
        tabulation[i] = row - row[pivot_col_idx] * pivot_row_

    visualize_tabulation(tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, big_M=True)

    # the values in the solution column except the one in the objective function row must be non-negative
    is_feasible = (tabulation[:, -1][2: ] >= 0).all()

    if not is_feasible:
        raise Exception(f"the tabulation is not feasible...")
    print(f"the tabulation is feasible.")

    print('\n')

# check for many solutions
non_basic_vars = list(set(var_symbol_arr) - set(basic_var_symbol_arr))
non_basic_vars.sort()
for non_basic_var in non_basic_vars:
    # find the objective row value for each non-basic variable
    print(non_basic_var, obj_func_row[var_symbol_arr.index(non_basic_var)])
    if obj_func_row[var_symbol_arr.index(non_basic_var)] == 0:
        print(f"WARNING: non-basic variable '{non_basic_var}' has a zero value in the objective row; therefore, MANY SOLUTIONS EXISTS...")

==================== iteration no.: 1 ====================
entering var: X2
pivot col: [Fraction(-20, 1) Fraction(-20, 1) Fraction(20, 1) Fraction(20, 1)]
solution : [Fraction(0, 1) Fraction(-20, 1) Fraction(15, 1) Fraction(20, 1)]
ratio col: [-0.    1.    0.75  1.  ]
leaving var : S1
pivot row: [Fraction(0, 1) Fraction(2, 1) Fraction(20, 1) Fraction(4, 1)
 Fraction(1, 1) Fraction(0, 1) Fraction(15, 1)]
new basic variables: ['P', 'P', 'X2', 'A2']
┌────┬─────┬──────┬──────┬──────┬──────┬──────┬───────┐
│    │   P │  X1  │   X2 │   X3 │  S1  │   A2 │  sol  │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ P  │   1 │ -4M  │    0 │ 14   │ 1+1M │    0 │ 15-5M │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ X2 │   0 │ 1/10 │    1 │  0.2 │ 1/20 │    0 │  3/4  │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ A2 │   0 │  4   │    0 │  0   │  -1  │    1 │   5   │
└────┴─────┴──────┴──────┴──────┴──────┴──────┴───────┘
the tabulation is feasible.


===============

/tmp/ipykernel_675409/4064744434.py:36: RuntimeWarning: divide by zero encountered in divide
  ratio_col = np.float64(solution) / np.float64(pivot_col)


- Now, we will define a function to remove aritificial variables from a given tabulation. 

In [7]:
def remove_artificial_var(tabulation, all_vars, basic_vars):

    tabulation = copy.deepcopy(tabulation)
    all_vars   = copy.deepcopy(all_vars)

    i = 0
    while i < tabulation.shape[1]-1:

        if all_vars[i].startswith('A'):
            # an artificial variable
            tabulation = np.concatenate((tabulation[:, range(i)], tabulation[:, range(i+1, tabulation.shape[1])]), axis=1)
            all_vars.remove(all_vars[i])
        else:
            i += 1

    # remove the big-M row
    tabulation = np.concatenate((tabulation[0].reshape(1, -1), tabulation[2: ]))
    basic_vars.remove(OBJ_FUNC_VAR_SYMBOL)

    return tabulation, all_vars, basic_vars

In [8]:
tabulation_1, var_symbol_arr_1, basic_var_symbol_arr_1 = remove_artificial_var(tabulation, var_symbol_arr, basic_var_symbol_arr)

In [9]:
visualize_tabulation(tabulation_1, all_vars=var_symbol_arr_1, basic_vars=basic_var_symbol_arr_1)

┌────┬─────┬──────┬──────┬──────┬────────┬────────┐
│    │   P │   X1 │   X2 │   X3 │     S1 │    sol │
├────┼─────┼──────┼──────┼──────┼────────┼────────┤
│ P  │   1 │    0 │    0 │ 14   │  1     │ 15     │
├────┼─────┼──────┼──────┼──────┼────────┼────────┤
│ X2 │   0 │    0 │    1 │  0.2 │  0.075 │  0.625 │
├────┼─────┼──────┼──────┼──────┼────────┼────────┤
│ X1 │   0 │    1 │    0 │  0   │ -0.25  │  1.25  │
└────┴─────┴──────┴──────┴──────┴────────┴────────┘


### Define a Function for Integer Programming

In [24]:
def integer_program(tabulation, all_vars, basic_vars, opt_type):

    def is_integer(tabulation):
        value = True
        for sol in tabulation[:, -1][1: ]:
            if sol.denominator != 1: value = False; break
        return value
    
    def extract_frac(arr):

        frac_arr = np.zeros([len(arr)], dtype=object)

        for i, real_num in enumerate(arr):
            frac_arr[i] = real_num - Fraction(int(real_num))
        
        return frac_arr

    tabulation = copy.deepcopy(tabulation)
    all_vars = copy.deepcopy(all_vars)
    basic_vars = copy.deepcopy(basic_vars)

    iteration_no = 0
    while not is_integer(tabulation):

        iteration_no += 1
        print("#"*30 + f" ITERATION NO. {iteration_no} " + "#"*30)

        # find the largest fraction in the solution column
        solution_frac_arr = extract_frac(tabulation[:, -1][1: ])
        max_frac = np.max(solution_frac_arr)
        print(f"max. frac: {max_frac}")

        # find the corresponding resource row (there can be multiple instances of `max_frac`)
        resource_row_idx = list(solution_frac_arr).index(max_frac) + 1
        print(f"resource var: {basic_vars[resource_row_idx]}")

        # create the constraint
        constraint = -extract_frac(tabulation[resource_row_idx])
        tabulation, all_vars, basic_vars = add_constraint(tabulation, constraint, new_var="G"+str(iteration_no), all_vars=all_vars, basic_vars=basic_vars)
        visualize_tabulation(tabulation, all_vars, basic_vars)
        
        # apply dual simplex method
        print("\nAPPLYING DUAL SIMPLEX...")
        tabulation, basic_vars = dual_simplex_to_all_slack_starting(tabulation, all_vars, basic_vars)
        
        # apply all-slack starting to make the tabulation optimal
        print("\nAPPLYING ALL-SLACK STARTNG...")
        tabulation, basic_vars = all_slack_starting(tabulation, all_vars, basic_vars, opt_type)

        visualize_tabulation(tabulation, all_vars, basic_vars)

        if iteration_no > 5: 
            print("the maximum no. of iteration is surpassed...")
            break

In [25]:
integer_program(tabulation_1, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, opt_type=OPT_TYPE)

############################## ITERATION NO. 1 ##############################
max. frac: 5/8
resource var: X2
┌─────┬──────┬──────┬──────┬──────┬────────┬──────┬────────┐
│  P  │   X1 │   X2 │   X3 │   S1 │     A2 │   G1 │    sol │
├─────┼──────┼──────┼──────┼──────┼────────┼──────┼────────┤
│  P  │    1 │    0 │    0 │ 14   │  1     │    0 │ 15     │
├─────┼──────┼──────┼──────┼──────┼────────┼──────┼────────┤
│ X2  │    0 │    0 │    1 │  0.2 │  0.075 │    0 │  0.625 │
├─────┼──────┼──────┼──────┼──────┼────────┼──────┼────────┤
│ X1  │    0 │    1 │    0 │  0   │ -0.25  │    0 │  1.25  │
├─────┼──────┼──────┼──────┼──────┼────────┼──────┼────────┤
│ G1  │    0 │    0 │    0 │ -0.2 │ -0.075 │    1 │ -0.625 │
└─────┴──────┴──────┴──────┴──────┴────────┴──────┴────────┘

APPLYING DUAL SIMPLEX...
====================DUAL SIMPLEX - iteration no.: 1====================
leaving variale : G1
ratio row       : [        inf         nan         nan 70.         13.33333333  0.
 24.        ]
ent

/home/akila/semester_8/OR/helper.py:152: RuntimeWarning: divide by zero encountered in divide
  ratio_row = np.float64(np.abs(tabulation[0])) / np.float64(np.abs(tabulation[pivot_row_idx]))
/home/akila/semester_8/OR/helper.py:152: RuntimeWarning: invalid value encountered in divide
  ratio_row = np.float64(np.abs(tabulation[0])) / np.float64(np.abs(tabulation[pivot_row_idx]))


ValueError: zero-size array to reduction operation minimum which has no identity

- The error is understandable; once the other resource row with the fraction 1/3 is selected, dual simplex method can be applied on the tabulation. 
- This feature is needed to be added to the above function. 